# importing Libraries

In [1]:
!pip install -U spacy-nightly --pre
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
2023-11-02 18:57:01.748681: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 18:57:01.748745: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register f

In [2]:
!pip install spacy-transformers

  Using cached transformers-4.30.2-py3-none-any.whl.metadata (113 kB)
  Using cached tokenizers-0.13.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (7.8 MB)
Using cached transformers-4.30.2-py3-none-any.whl (7.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [16]:
import requests
import json
import pandas as pd

import spacy
from spacy import displacy
import spacy_transformers

import os
import hashlib

# Downloading Data
I am using the world bank annual data report for this purpose of making knowledge graphs

In [4]:
url = "https://documents1.worldbank.org/curated/en/099092823161580577/text/BOSIB055c2cb6c006090a90150e512e6beb.txt"
response = requests.get(url)

if response.status_code == 200:
    with open("downloaded_file.txt", "w", encoding="utf-8") as file:
        file.write(response.text)
    print("File downloaded successfully.")
else:
    print(f"Failed to download file. Status code: {response.status_code}")

File downloaded successfully.


# Applying the Entity relationship pair on it

In [5]:
!python3 -m spacy download en_core_web_trf

2023-11-02 18:58:39.209162: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 18:58:39.209228: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 18:58:39.209282: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 18:58:40.588100: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 4.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [17]:
eng_nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [6]:
# Load the spacy transformer (roberta-base) model
roberta_nlp = spacy.load("en_core_web_trf")

In [7]:
def return_entities(pipeline, texts):
    # List to store entity text and label pairs
    entities_list = []
    document = pipeline(texts)

    # Entity text & label extraction
    for entity in document.ents:
        entities_list.append((entity.text, entity.label_))

    # You may also want to render each document's entities using displacy
    displacy.render(document, jupyter=True, style='ent')

    return entities_list

Loading the text file

In [8]:
file_path = "downloaded_file.txt"

with open(file_path, "r", encoding="utf-8") as file:
    file_content = file.read()

# Now, 'file_content' contains the content of the file
print(file_content)

A New Era
in Development
ANNUAL REPORT 2023
Contents
	 3	Introduction
	 4	 Message from the President
	 6	 Message from the Executive Directors
	 9	 Addressing a Multitude of Global Challenges
	 15	 Regional Engagements
	45	 Working to Achieve Development Goals
	 71	 Promoting Sustainable Finance and Capital Markets
	 75	 Engaging with Partners on Shared Priorities
	 78	 Enhancing Effectiveness and Sustainability in Our Operations
	 81 	 Our Values, People, and Places
	 87	 The World Bank’s Climate-Related Disclosures
	89	 Guiding the Institution
	90 	 Providing Oversight and Accountability
	94 	 Deploying Resources Strategically
	106 	 Commitment to Results

KEY TABLES
	
100 	IBRD Key Financial Indicators, Fiscal 2019–23
	
104 	IDA Key Financial Indicators, Fiscal 2019–23


This annual report, which covers the period from July 1, 2022, to June 30, 2023,
has been prepared by the Executive Directors of both the International Bank for
Reconstruction and Development (IBRD) and the Inter

## Loading the entities

In [9]:
list_entities=return_entities(roberta_nlp,file_content)

In [10]:
list_entities

[('4', 'CARDINAL'),
 ('The World Bank’s', 'ORG'),
 ('Fiscal 2019–23\n\t\n', 'DATE'),
 ('Fiscal 2019–23', 'DATE'),
 ('annual', 'DATE'),
 ('the period from July 1, 2022,', 'DATE'),
 ('June 30, 2023', 'DATE'),
 ('the International Bank for\nReconstruction and Development', 'ORG'),
 ('IBRD', 'ORG'),
 ('the International Development\n', 'ORG'),
 ('the World Bank', 'ORG'),
 ('two', 'CARDINAL'),
 ('Ajay Banga', 'PERSON'),
 ('the World\n', 'ORG'),
 ('the Board of Executive Directors', 'ORG'),
 ('the Board of Governors', 'ORG'),
 ('Annual', 'DATE'),
 ('World Bank Group', 'ORG'),
 ('the International\nFinance Corporation', 'ORG'),
 ('IFC', 'ORG'),
 ('the Multilateral Investment Guarantee Agency', 'ORG'),
 ('MIGA', 'ORG'),
 ('the International Centre for Settlement of Investment Disputes', 'ORG'),
 ('World Bank', 'ORG'),
 ('Bank', 'ORG'),
 ('World Bank Group', 'ORG'),
 ('Bank Group', 'ORG'),
 ('five', 'CARDINAL'),
 ('U.S.', 'GPE'),
 ('IBRD', 'ORG'),
 ('IDA Financial\nStatements and Management’s D

In [ ]:
unique_second_items = set(item[1] for item in list_entities)

print(unique_second_items)

## Loading Entities and relations

Getting all the unqiue entities

In [20]:
from spacy.tokens import Doc

Doc.set_extension('rel', default={}, force=True)

In [21]:
def extract_relations(documents,nlp,nlp2):
  predicted_rels = list()
  for doc in nlp.pipe(documents, disable=["tagger", "parser"]):
    source_hash = hashlib.sha256(doc.text.encode('utf-8')).hexdigest()
    for name, proc in nlp2.pipeline:
          doc = proc(doc)

    for value, rel_dict in doc._.rel.items():
      for e in doc.ents:
        for b in doc.ents:
          if e.start == value[0] and b.start == value[1]:
            max_key = max(rel_dict, key=rel_dict. get)
            #print(max_key)
            e_id = hashlib.sha256(str(e).encode('utf-8')).hexdigest()
            b_id = hashlib.sha256(str(b).encode('utf-8')).hexdigest()
            if rel_dict[max_key] >=0.9 :
              #print(f" entities: {e.text, b.text} --> predicted relation: {rel_dict}")
              predicted_rels.append({'head': e_id, 'tail': b_id, 'type':max_key, 'source': source_hash})
  return predicted_rels

In [ ]:
predicted_rels = extract_relations(file_content,eng_nlp,roberta_nlp)

# Making Knowledge Graphs

In [11]:
from neo4j import GraphDatabase

In [12]:
class Neo4jConnection:

    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)

    def close(self):
        if self.__driver is not None:
            self.__driver.close()

    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.__driver.session(database=db) if db is not None else self.__driver.session()
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

Setting up the connection

In [ ]:
host = 'bolt://44.203.34.34:7687'
user = 'neo4j'
password = 'drawings-rattle-funds'
driver = GraphDatabase.driver(host,auth=(user, password))

In [ ]:
def neo4j_query(query, params=None):
  '''A function for querying the neo4j graph in which we can update the value, see reomve the values'''
  with driver.session() as session:
      result = session.run(query, params)
      return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [ ]:

neo4j_query("""
MATCH (n) DETACH DELETE n;
""")

The first NODE which in my case I am making it as world bank data

In [ ]:
neo4j_query("""
MERGE (l:WorldBank {name:"World Bank"})
RETURN l
""")

In [ ]:
neo4j_query("""
MATCH (l:WorldBank)
UNWIND $data as row
MERGE (o:Offer{id:row.text_sha256})
SET o.text = row.text
MERGE (l)-[:HAS_OFFER]->(o)
WITH o, row.annotations as entities
UNWIND entities as entity
MERGE (e:Entity {id:entity.id})
ON CREATE SET
              e.name = entity.text,
              e.label = entity.label_upper
MERGE (o)-[m:MENTIONS]->(e)
ON CREATE SET m.count = 1
ON MATCH SET m.count = m.count + 1
WITH e as e
CALL apoc.create.addLabels( id(e), [ e.label ] )
YIELD node
REMOVE node.label
RETURN node
""", {'data': list_entities})